In [ ]:
# Setting up a custom stylesheet in IJulia
file = open("style.css") # A .css file in the same folder as this notebook file
styl = read(file, String) # Read the file
HTML("$styl") # Output as HTML

# Data

<h2>In this lesson</h2>

- [Introduction](#Introduction)
- [Importing the packages for this lesson](#Importing-the-packages-for-this-lesson)
- [Outcome](#Outcome)
- [The standard normal distribution](#The-standard-normal-distribution)
- [Using the Distributions package](#Using-the-Distributions-package)
- [Comparing samples](#Comparing-samples)
- [Plotly](#Plotly)

<hr>
<h2>Introduction</h2>

Data is everywhere and working with data is very important in scientific computing.  Data point values come in patterns, called distributions.  Whether you are modelling, getting data from a research project, or simply want ot manufactor some data, Julia is the place to go.

We know about the `rand()` and `randn()` functions, but by using the `Distributions` package we can do much, much more.

[Back to the top](#In-this-lesson)

<hr>
<h2>Importing the packages for this lesson</h2>

In [ ]:
import Pkg

Pkg.add("Distributions")
Pkg.add("StatsPlots")
Pkg.add("HypothesisTests")
Pkg.add("GLM")
Pkg.add("ORCA")
Pkg.add("Plots")
Pkg.add("DataFrames")

In [ ]:
using Distributions
using DataFrames
using StatsPlots
using HypothesisTests
using GLM 
using ORCA
using Plots

Initiate GR backend for plotting

In [ ]:
gr()

<hr>
<h2>Outcome</h2>

After successfully completing this lecture, you will be able to:

- Understand and plot the standard normal distribution
- Create a variety of random value from different distributions using the `Distributions` package
- Plot some of the distributions in the `Distributions` package
- Create your own data using a distribution and its parameters
- Use the online plotting library `Plotly`

<hr>
<h2>The standard normal distribution</h2>

The Julie function `rand()` allows us to return an array of randomly selected values.  The values come from a specific distribution, namely the **standard normal** distribution.  The majority of values cluster around the mean of $ 0 $ and a standard deviation of $ 1 $.  Let's get $ 1000 $ such values and attach this array to the variable `norm1`.

In [ ]:
# We will put a semicolon at the end to supress the output
norm1 = randn(1000);

We can plot this as histogram.  In the example below, we will use the keyword argument `bins`.  Setting it to $ 10 $ means that between the minimum and maximum value we create $ 10 $ equally sized ranges and count how many values occur in each range.

In [ ]:
# Beware the pink text box!!!
Plots.histogram(norm1, bins = 10, label = "Standard normal distribution", title = "Histogram")

These values were selected at random.  We can check how close we came to a real mean of $ 0 $ and a standard deviation of $ 1 $.

In [ ]:
# Using mean()
mean(norm1)

In [ ]:
# Standard deviation using std()
std(norm1)

[Back to the top](#In-this-lesson)

<hr>
<h2>Using the Distributions package</h2>

We can ask for random data point values for a variable to be taken from any number of discrete or continuous distributions.  This greatly expands on the standard normal distribution provided by the `randn()` function.  In this lesson we will concentrate on the continuous random variables and start with the normal distribution.

### The normal distribution

The `Normal()` function from the Distribution package takes two arguments.  The first is the mean and the second is the required standard deviation.  We use it in conjunction with the `rand()` function so that we can specifiy how many data point values we want.

In [ ]:
# Recreating the standard normal distribution
norm2 = rand(Normal(0, 1), 1000);

In [ ]:
# Histogram
Plots.histogram(norm2, bins = 10, label = "Standard normal distribution", title = "Histogram")

Just to belabor the point, we can use `Plots` to show us the theoretical normal distribution.

Let's use plotlyjs as our plots backend from here.

In [ ]:
plotlyjs()

In [ ]:
Plots.plot(Normal(0, 1), fill = (0.5, :blue), label = "Standard normal distribution",
title = "Density")

We can also fit some data to a distribution.  In the example below we use the `norm1` array and fit it to the standard normal distribution.

In [ ]:
fit(Normal, norm1)

We can plot other distributions as well.  Here is the $ {\chi}^{2} $ distribution with different degrees of freedom.

In [ ]:
Plots.plot(Chisq(3), fill = (0.25, :blue), label = "3 degrees of freedom")
Plots.plot!(Chisq(5), fill = (0.25, :orange), label = "5 degrees of feedom")
Plots.plot!(Chisq(10), fill = (0.25, :deepskyblue), label = "10 degrees of freedom")

The exponential distribution with scale parameter $ \theta $.

In [ ]:
Plots.plot(Exponential(2), fill = (0.25, :blue), label = "Scale parameter 2")
Plots.plot!(Exponential(4), fill = (0.25, :orange), label = "Scale parameter 4")
Plots.plot!(Exponential(6), fill = (0.25, :deepskyblue), label = "Scale parameter 6")

[Back to the top](#In-this-lesson)

<hr>
<h2>Comparing samples</h2>

The reason for the detour to a quick peek into the distributions package, is that we want to create arrays of data point values or elements that are picked at random, each drawn from a specific distribution.

So, imagine that we compare the results of two sets of experiments or observed outcomes.  These can take on many, many form.  We have seen data from an Ebola epidemic, but we can consider a lot more.  Let's keep it simple and consider the examination results of a particular course.  Imagine that we have the results of two consecutive years.  Our aim is to learn how to use Julia.  So, we don't need actual results.  We can simply simulate some results.  You can well imagine that these values can be stored as arrays!

If $ 100 $ students take the examination each year, we can use the `rand()` function to generate random values.  As argument, though, we pass a specific distribution.  We'll use the normal distribution with a mean and a standard deviation.

In [ ]:
year1 = rand(Normal(67, 10), 100)
year2 = rand(Normal(71, 15), 100);

Our aim will be to compare these.  The average scored for each year differs.  Are they statistically different, though?  This is not a course in statistics, but using Julia, we'll see how easy it is to tell us if there is such a difference.

For now, let's plot the examination results as theoretical distributions.

In [ ]:
Plots.plot(Normal(67, 10), fill = (0.5, :orange), label = "Year 1", title = "Boxplot")
Plots.plot!(Normal(71, 15), fill = (0.5, :blue), label = "Year 2")

We can get more statistical information from these two distributions.

In [ ]:
# Skewness
skewness(year1), skewness(year2)

In [ ]:
# Kurtosis
kurtosis(year1), kurtosis(year2)

Now let's compare these two year by way of Student's *t*-test.

In [ ]:
EqualVarianceTTest(year1, year2)

Imagine instead that these were the same students going on from year $ 1 $ to year $ 2 $.  We want to know if there is a correlation between the marks in the two years.

In [ ]:
data = DataFrame(One = year1, Two = year2);

In [ ]:
first(data, 5)

In [ ]:
OLS = glm(@formula(One ~ Two), data, Normal(), IdentityLink())

[Back to the top](#In-this-lesson)

## Plotly

Plotly is a collabrotaive / social website for plotting.  Let's have a look.

In [ ]:
Plots.plot(Plots.fakedata(50, 5), w = 1)

[Back to the top](#In-this-lesson)